# Mutaciones y análisis de Brauer
El siguiente programa computa las semillas formuladas a partir del siguiente vector de polinomios  
$S_0 := {(1,x),(1+x^2, 1+x+x^2)$ con todos los posibles vectores del campo $\mathbb{F}:= [\mathbb{Z_2}]/<x^3+x+1>$, comparando asi la vida de cada semilla generada.

In [99]:
from sympy import symbols, Poly, GF, div
import numpy as np

# Definir el símbolo y el campo finito Z_2
x = symbols('x')

# Definición del campo Z_2 y divisor
campo_finito = GF(2)
divisor = x ** 3 + x + 1


In [100]:
# Función para la representación binaria
def poly_a_binario(poly):
    # Obtener los coeficientes del polinomio
    coeficientes = poly.all_coeffs()

    # Convertir los coeficientes a binario y unirlos en una cadena
    representacion_binaria = ''.join([str(int(coeficiente)) for coeficiente in coeficientes])

    # Rellenar con ceros a la izquierda si es necesario
    representacion_binaria = representacion_binaria.zfill(3)

    return representacion_binaria


# Función para definir la funcion T(w(sk-1))
def T_w_sk_1(matrix_A, omega_1, vs):
    matrix_A_w1 = matrix_A.dot(omega_1)
    return matrix_A_w1 + vs


# Función para calcular w_0 + T(w(sk-1)) 
def omega_prime(omega_0, T_w):
    return omega_0 + T_w


# Reducción del polinomio dado el divisor 
def residuo(w, divisor):
    mutacion_n = []
    for i in w:
        for polinomio in i:
            cociente, res = div(polinomio, divisor, domain=campo_finito)
            mutacion_n.append([res])
    return np.array(mutacion_n)



In [101]:
# Definición de los polinomios de la tarea

polinomio_1 = Poly(1, x, domain=campo_finito)
polinomio_2 = Poly(x, x, domain=campo_finito)

polinomio_3 = Poly(1 + x ** 2, x, domain=campo_finito)
polinomio_4 = Poly(1 + x + x ** 2, x, domain=campo_finito)

w_0 = np.array([[polinomio_1], [polinomio_2]])
w_1 = np.array([[polinomio_3], [polinomio_4]])

print(f'Elementos de la semilla original')
print(f'x_0 = ({polinomio_1.as_expr(), polinomio_2.as_expr()} {polinomio_3.as_expr(), polinomio_4.as_expr()})')
print(f'w(0) = {poly_a_binario(polinomio_1) + poly_a_binario(polinomio_2)}')
print(f'w(1) = {poly_a_binario(polinomio_3) + poly_a_binario(polinomio_4)}')

Elementos de la semilla original
x_0 = ((1, x) (x**2 + 1, x**2 + x + 1))
w(0) = 001010
w(1) = 101111


In [102]:
# Definición de la matrix A
polinomio_5 = Poly(1 + x, x, domain=campo_finito)
polinomio_6 = Poly(x + x ** 2, x, domain=campo_finito)
matrix_A = np.array([[polinomio_2, polinomio_5], [polinomio_6, polinomio_1]])

# Definimos el vs a mirar
polinomio_7 = Poly(0, x, domain=campo_finito)
vs = np.array([[polinomio_7], [polinomio_7]])

In [103]:
i = 1
w_0_copy = w_0
w_1_copy = w_1
while i > 0:
    T_w = T_w_sk_1(matrix_A=matrix_A, omega_1=w_1_copy, vs=vs)
    w_i = omega_prime(omega_0=w_0_copy, T_w=T_w)
    mutacion_i = residuo(w_i, divisor)

    w_0_copy = w_1_copy
    w_1_copy = mutacion_i

    print(f'Elementos de la mutación {i}')
    print(
        f'x_{i} = ({w_0_copy[0, 0].as_expr(), w_0_copy[1, 0].as_expr()}, {w_1_copy[0, 0].as_expr(), w_1_copy[1, 0].as_expr()})')
    print(f'w(0) = {poly_a_binario(w_0_copy[0, 0]) + poly_a_binario(w_0_copy[1, 0])}')
    print(f'w(1) = {poly_a_binario(w_1_copy[0, 0]) + poly_a_binario(w_1_copy[1, 0])}')
    i += 1
    if np.array_equal(w_0_copy, w_0) and np.array_equal(w_1_copy, w_1):
        break

print(i)

Elementos de la mutación 1
x_1 = ((x**2 + 1, x**2 + x + 1), (x, x**2 + x))
w(0) = 101111
w(1) = 010110
Elementos de la mutación 2
x_2 = ((x, x**2 + x), (0, x**2 + x))
w(0) = 010110
w(1) = 000110
Elementos de la mutación 3
x_3 = ((0, x**2 + x), (x + 1, 0))
w(0) = 000110
w(1) = 011000
Elementos de la mutación 4
x_4 = ((x + 1, 0), (x**2 + x, x**2 + x + 1))
w(0) = 011000
w(1) = 110111
Elementos de la mutación 5
x_5 = ((x**2 + x, x**2 + x + 1), (x**2 + x, x**2 + 1))
w(0) = 110111
w(1) = 110101
Elementos de la mutación 6
x_6 = ((x**2 + x, x**2 + 1), (x**2 + 1, 0))
w(0) = 110101
w(1) = 101000
Elementos de la mutación 7
x_7 = ((x**2 + 1, 0), (x**2 + x + 1, x**2 + x))
w(0) = 101000
w(1) = 111110
Elementos de la mutación 8
x_8 = ((x**2 + x + 1, x**2 + x), (1, x))
w(0) = 111110
w(1) = 001010
Elementos de la mutación 9
x_9 = ((1, x), (x + 1, x))
w(0) = 001010
w(1) = 011010
Elementos de la mutación 10
x_10 = ((x + 1, x), (1, 1))
w(0) = 011010
w(1) = 001001
Elementos de la mutación 11
x_11 = ((1, 1)